Fossil Fuel Power Plants vs Renewable Power Plants

Comparing environmental, health and economic data by county.

- Compare Towns with Nuclear vs Towns without Nuclear
- Compare Towns with renewables within themselves
- Compare Towns with a power plant vs the rest of the state